In [ ]:
import random
import numpy as np
import skimage as sk
from matplotlib import pyplot as plt

# this function returns the number of black pixels in the image
def countTheBlack(image, col = 0):
    return np.sum(image == col)


## Open target image and make some transforms to the image
## IMG vairable is the target image
IMG = sk.io.imread('./Circle.png', as_gray=True)
IMG = sk.transform.resize(IMG, (500, 500))
IMG = IMG > 0.5
IMG = IMG.astype(np.uint8) * 255
IMG = np.pad(IMG, ((25, 25), (25, 25)), mode='constant', constant_values=255)

print("Initial Black Count: ", countTheBlack(IMG))


## Draw circle 
rr,cc = sk.draw.circle_perimeter(IMG.shape[0] // 2, IMG.shape[1] // 2, IMG.shape[0] // 2 - 1)
IMG[rr, cc] = 128


## This script extracts the edges from the target image
## store detected edges in the edges array vairable
edges = sk.feature.canny(
    image=IMG,
    sigma=2,
    low_threshold=0,
    high_threshold=0.5
)
edges = edges == 0      ## Make detected edges 0 (black)
IMG[edges == 0] = 87    ## make detected edges pixels value 87. (As label)

print("Black Count after edge: ", countTheBlack(IMG))
print("Edge Count: ", countTheBlack(IMG, 87))

## Show results
FullBlackCount = countTheBlack(IMG)
FullEdgeCount = countTheBlack(IMG, 87)
plt.imshow(IMG, cmap='gray')
plt.show()
plt.imshow(edges, cmap='gray')
plt.show()

In [ ]:
## This function returns point (x,y) coordinates
def getCircleCords(num):
    radians = np.radians(num)
    x = int((np.cos(radians) * IMG.shape[1] / 2 + IMG.shape[1] / 2)) -1
    y = int((np.sin(radians) * IMG.shape[0] / 2+ IMG.shape[0] / 2)) -1
    return x, y

## This function takes two points and
## draw line between them on solution image.
def drawALine(num1, num2, image):
    x1, y1 = getCircleCords(num1)
    x2, y2 = getCircleCords(num2)
    rr, cc = sk.draw.line(x1, y1, x2, y2)
    image[rr, cc] = 128


In [ ]:
## Three hyperparameters used : POPULATION_SIZE, MUTATION_RATE, CROSSOVER_RATE 
POPULATION_SIZE = 500       ## Populasyon büyüklüğü. Number of individuals.
MUTATION_RATE = 0.01         ## Mutasyon oranı. yeni bireyin bazı değerleri değişim oranı
CROSSOVER_RATE = 0.9         ## Crossover oranı. çaprazlama ile yeni birey oluşturma oranı
GENLEN = 100                 ## Gene length. kullanılacak nokta sayısı
GenerationNumber = 5000       ## Number of generations. (nesil sayısı)
ExpectedSuccessRate = 0.8    ## This vairable used to stop generation if fitness score is higher than 0.8
EDGEIMPORTANCE = 0.5

In [ ]:
## This class used to represent each individual (solution) in the population.
## This class contains methods used on individuals
class Individual(object):
	## instance's (individual) attributes definition.
	def __init__(self, chromosome):
		self.chromosome = chromosome
		self.fitness = self.cal_fitness(0)
	
	## This method create random gene.
	## Select random number in range (0,360) 
	@classmethod
	def mutated_genes(self):
		gene = random.randint(0, 359)
		return gene
	
	## This method create chromosome with random genes.
	@classmethod
	def create_gnome(self):
		return [self.mutated_genes() for _ in range(GENLEN)]
	
	## This method create new chromosome with crossover operator.
	## Select from which chromosome the gene will be taken or mutate the gene.
	## Parents: current chromosome and another one given as parameter (par2).
	## Returns new Individual.
	def mate(self, par2):
		global MUTATION_RATE
		child_chromosome = []		## New chromosome
		## To decide that each gene will be selected from parent1 (current chromosome)
		## or from parent2 (par2) or it will be mutated.
		for gp1, gp2 in zip(self.chromosome, par2.chromosome):
			prob = random.random() 				## Random number (0-1)
			parRate = (1 - MUTATION_RATE) / 2	## Parent selection rate.
			if prob < parRate:					## Take gene from current chromosome
				child_chromosome.append(gp1)
			elif prob < parRate * 2:			## Take gene from par2
				child_chromosome.append(gp2)
			else:								## Select random gene
				child_chromosome.append(self.mutated_genes())

		return Individual(child_chromosome)
	
	## Here is two fitness function.
	## The first one is sufficient for use.
	def cal_fitness(self,fun = 0):
		global IMG,FullBlackCount,FullEdgeCount
		image = np.copy(IMG)
		i=0
		## Draw a line between points in the chromosome.
		for j in range(1,len(self.chromosome)):
			drawALine(self.chromosome[i], self.chromosome[j],image)
			i = j
		## To choose first fitness function.
		## Returns fitness score considering rate of black pixels in the drawed image.
		## FullBlackCount is the target image black pixels number.
		if fun == 0:
			return (1 - countTheBlack(image) / FullBlackCount)
		## Second fitness function.
		## Calculates edge rate.
		## Returns fitness score considering edge rate and black rate depending in the edge importance
		else:
			edgeRate = (countTheBlack(image, 87) / FullEdgeCount) * EDGEIMPORTANCE
			blackRate = (countTheBlack(image) / FullBlackCount) * (1 - EDGEIMPORTANCE)
			return (1 - edgeRate - blackRate)
		
	## This function used to draw the image.
	def drawTheImage(self,z =-1):
		global IMG,rr,cc
		image = np.zeros(IMG.shape)		## Create new white image.
		i = 0
		image[rr, cc] = 128		## Draw circle
		## Draw a line between points in the chromosome.
		for j in range(1,len(self.chromosome)):
			drawALine(self.chromosome[i], self.chromosome[j], image)
			i = j
		plt.imshow(image, cmap='gray')
		plt.show()
		if z != -1 and False:
			image = image.astype(np.uint8) * 255
			image = sk.color.gray2rgb(image)
			sk.io.imsave(f'./Images/Candy-1/{z}.jpg',image)

In [ ]:
def main():
	global POPULATION_SIZE,CROSSOVER_RATE,ExpectedSuccessRate,FullBlackCount,GenerationNumber
	generation = 1			## Current generation number.
	noChange = 0			## To count the number of successive iterations that has no enhance on fitness score.
	found = False			## If True then the desired solution is found.
	population = []			## Population array to store individuals.
	pastFitness = 0			## Stores the past fitness.
	fitnesses = []			## Stores fitnesses during process.

	fitnesses.append(0) ## Initialize first fitness score.
	
	## Create Population with random chromosomes.
	for _ in range(POPULATION_SIZE):
				gnome = Individual.create_gnome()
				population.append(Individual(gnome))

	## Main loop that continues until found solution with expected success rate or no change for 100 generations.
	while found == False and generation <= GenerationNumber:
		# Sort the population in descending order by fitness score.
		population = sorted(population, key = lambda x:x.fitness , reverse=True)

		best = population[0].fitness		## Store individual best fitness score.
		## Check current fitness score with previous. update if not equal
		## else, increament no change counter. 
		if (best != pastFitness):
			pastFitness = best		## Update the fitness score.
			# Print the current generation, the best fitness and draw the image.
			print("Generation: {}\tFitness: {}".\
		  		format(generation,best))
			population[0].drawTheImage()
			noChange = 0		## Reset the no change counter
		else:
			noChange += 1		## Increament counter

		## Stop loop if one of termination criterions met
		if best >= ExpectedSuccessRate or noChange > 100:
			found = True
		else:
			## Else generate generation list for new chromosomes.
			new_generation = []
			## The number of chromosomes which will be inserted from population.
			s = int((1 - CROSSOVER_RATE) * POPULATION_SIZE)
			new_generation.extend(population[:s])
			## Number of chromosomes which will be generated by cross over.
			s = int(CROSSOVER_RATE * POPULATION_SIZE)
			## Selects two random chromosomes from population.
			## Make crossing over operator on them to create new chromosomes.
			for _ in range(s):
				parent1 = random.choice(population[:50])
				parent2 = random.choice(population[:50])
				child = parent1.mate(parent2)
				new_generation.append(child)
			
			fitnesses.append(pastFitness)		## Save fitness scores.
			## This process deletes all previous chromosomes.
			## Replace them with new generated chromosomes.
			population = new_generation
			generation += 1			## Increament generation number.

	## End of loop.
	
	if generation-1 == GenerationNumber:
		print("Process terminated due to expired time.")
	else:
		print(f"Process terminated due to reaching desired result after {generation} generation.")
	## Print the best solution.
	print("Best result:")
	population[0].drawTheImage()
	print("-------------------------------------------------------")

	plt.plot(range(0,generation),fitnesses,linewidth = 2)
	plt.title("Population fitness scores")
	plt.show()

if __name__ == '__main__':
	main()